In [1]:
import matplotlib.pyplot as plt
import numpy as np
from custom_sepsis import *

In [2]:
ts = [{
    Simplification.SIMPLE: DirThompsonSampling(create_model(Simplification.SIMPLE), {},{}, {}, f"Simple-{i}", {}),
    Simplification.MEDIUM: DirThompsonSampling(create_model(Simplification.MEDIUM), {},{}, {}, f"Medium-{i}", {}),
    Simplification.NONE: FullThompsonSampling(create_model(Simplification.NONE), {},{}, {}, f"None-{i}", {}),
} for i in range(3)]
policies = [{
    Simplification.SIMPLE: random_policy(),
    Simplification.MEDIUM: random_policy(),
    Simplification.NONE: random_policy(),
} for i in range(3)]
rewards = {
    Simplification.SIMPLE: {},
    Simplification.MEDIUM: {},
    Simplification.NONE: {},
}
model_types = [Simplification.NONE] #[Simplification.SIMPLE, Simplification.MEDIUM, Simplification.NONE]

In [3]:
nr_episodes = 10000


In [ ]:
V = np.zeros(n_states)
for i in range(1, nr_episodes+1):
    for key in range(3):
        if (np.log2(i) % 1 == 0 and i<100) or (i % 100 == 0): # every power of 2 or 100 episodes, a new policy is computed
            for model_type in model_types:
                state_counts = ts[key][model_type].model.get_state_counts()
                policy, V = matrix_value_iteration(V, ts[key][model_type].model.transition_model())
                ts[key][model_type].add_data(i, rewards[model_type], policy, state_counts)
                ts[key][model_type].save()
                rewards[model_type] = {}
        for model_type in model_types:
            episode = run_episode(policies[key][model_type])
            rewards[model_type][i] = np.sum(episode.rewards)
            ts[key][model_type].model.update_state_counts(episode)
            